In [96]:
# Dataset from: https://www.kaggle.com/datasets/parisrohan/credit-score-classification?select=train.csv

In [97]:


import tensorflow as tf
import pandas as pd
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt



In [98]:
df_train = pd.read_csv("train.csv")
df_train.shape

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(100000, 28)

In [99]:
df_train.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Monthly_Inhand_Salary,84998.0,4194.170850,3183.686167,303.645417,1625.568229,3093.745000,5957.448333,15204.633333
Num_Bank_Accounts,100000.0,17.091280,117.404834,-1.000000,3.000000,6.000000,7.000000,1798.000000
Num_Credit_Card,100000.0,22.474430,129.057410,0.000000,4.000000,5.000000,7.000000,1499.000000
Interest_Rate,100000.0,72.466040,466.422621,1.000000,8.000000,13.000000,20.000000,5797.000000
Delay_from_due_date,100000.0,21.068780,14.860104,-5.000000,10.000000,18.000000,28.000000,67.000000
Num_Credit_Inquiries,98035.0,27.754251,193.177339,0.000000,3.000000,6.000000,9.000000,2597.000000
Credit_Utilization_Ratio,100000.0,32.285173,5.116875,20.000000,28.052567,32.305784,36.496663,50.000000
Total_EMI_per_month,100000.0,1403.118217,8306.041270,0.000000,30.306660,69.249473,161.224249,82331.000000


In [100]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [101]:
df_train.isna().sum()

ID                              0
Customer_ID                     0
Month                           0
Name                         9985
Age                             0
SSN                             0
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary       15002
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                11408
Delay_from_due_date             0
Num_of_Delayed_Payment       7002
Changed_Credit_Limit            0
Num_Credit_Inquiries         1965
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      4479
Payment_Behaviour               0
Monthly_Balance              1200
Credit_Score                    0
dtype: int64

In [102]:
df_train = df_train.dropna()

In [103]:
# get rid of columns that don't help
df_train.drop(['ID', 'Month', 'Customer_ID', 'Name', 'SSN'], axis=1, inplace=True)

In [104]:
df_train.head(50)

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
6,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,...,Good,809.98,22.537593,22 Years and 7 Months,No,49.574949,178.3440674122349,Low_spent_Small_value_payments,244.5653167062043,Good
8,28_,_______,34847.84,3037.986667,2,4,6,1,Credit-Builder Loan,3,...,Good,605.03,24.464031,26 Years and 7 Months,No,18.816215,104.291825168246,Low_spent_Small_value_payments,470.69062692529184,Standard
9,28,Teacher,34847.84,3037.986667,2,4,6,1,Credit-Builder Loan,7,...,Good,605.03,38.550848,26 Years and 8 Months,No,18.816215,40.39123782853101,High_spent_Large_value_payments,484.5912142650067,Good
10,28,Teacher,34847.84_,3037.986667,2,1385,6,1,Credit-Builder Loan,3,...,_,605.03,33.224951,26 Years and 9 Months,No,18.816215,58.51597569589465,High_spent_Large_value_payments,466.46647639764313,Standard
12,28,Teacher,34847.84,3037.986667,2,4,6,1,Credit-Builder Loan,3,...,Good,605.03,34.977895,26 Years and 11 Months,No,18.816215,130.11542024292334,Low_spent_Small_value_payments,444.8670318506144,Good
13,28,Teacher,34847.84,3037.986667,2,4,6,1,Credit-Builder Loan,3,...,Good,605.03,33.381010,27 Years and 0 Months,No,18.816215,43.477190144355745,High_spent_Large_value_payments,481.505261949182,Good
15,28,Teacher,34847.84,3037.986667,2,4,6,1,Credit-Builder Loan,3,...,Good,605.03,32.933856,27 Years and 2 Months,No,18.816215,218.90434353388733,Low_spent_Small_value_payments,356.07810855965045,Good
16,34,_______,143162.64,12187.220000,1,5,8,3,"Auto Loan, Auto Loan, and Not Specified",5,...,Good,1303.01,28.616735,17 Years and 9 Months,No,246.992319,168.413702679309,!@9#%8,1043.3159778669492,Good
20,34,_______,143162.64,12187.220000,1,5,8,3,"Auto Loan, Auto Loan, and Not Specified",10,...,Good,1303.01,31.376150,18 Years and 1 Months,No,246.992319,430.9475278803298,Low_spent_Large_value_payments,810.7821526659284,Good


In [105]:
## Nominal Data - Occupation, Type_of_Loan, Payment_Behaviour
## Continuous Value - Annual_Income, Monthly_Inhand_Salary, Changed_Credit_Limit, Outstanding_Debt, 
    # Credit_Utilization_Ratio, Total_EMI_per_month, Amount_invested_monthly, Monthly_Balance
## Counts - Num_Bank_Accounts, Num_Credit_Card, Num_of_Loan, Delay_from_due_date, Num_of_Delayed_Payment, Num_Credit_Inquiries
## Integer - Interest_Rate
## Ordinal - Credit_Mix, 
## Binary - Payment_of_Min_Amount
## <<to parse>> - Credit_History_Age

## <<Prediction Label - Credit_Score (ordinal)

In [106]:
def convert_to_float(feature, df):
  df = df[pd.to_numeric(df[feature], errors='coerce').notnull()]
  df[feature] = df[feature].astype(float, errors = 'raise')
  return df

In [107]:
# Convert Annual_Income, Monthly_Inhand_Salary, Changed_Credit_Limit, Outstanding_Debt, 
    # Credit_Utilization_Ratio, Total_EMI_per_month, Amount_invested_monthly, Monthly_Balance
    # to float

df_train = convert_to_float('Age', df_train)
df_train = convert_to_float('Annual_Income', df_train)
df_train = convert_to_float('Monthly_Inhand_Salary', df_train)
df_train = convert_to_float('Changed_Credit_Limit', df_train)
df_train = convert_to_float('Outstanding_Debt', df_train)
df_train = convert_to_float('Credit_Utilization_Ratio', df_train)
df_train = convert_to_float('Total_EMI_per_month', df_train)
df_train = convert_to_float('Amount_invested_monthly', df_train)
df_train = convert_to_float('Monthly_Balance', df_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [140]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42493 entries, 0 to 99996
Columns: 6282 entries, Age to Payment_of_Min_Amount_Yes
dtypes: float64(10), int64(7), object(2), uint8(6263)
memory usage: 260.3+ MB


In [109]:
print(df_train['Occupation'].value_counts())

_______          3034
Lawyer           2903
Mechanic         2817
Teacher          2764
Architect        2735
Doctor           2733
Engineer         2724
Accountant       2722
Developer        2720
Media_Manager    2695
Entrepreneur     2694
Scientist        2674
Musician         2584
Journalist       2569
Manager          2533
Writer           2524
Name: Occupation, dtype: int64


In [110]:
print(df_train['Type_of_Loan'].value_counts())

Not Specified                                                                                                                                       719
Personal Loan                                                                                                                                       653
Credit-Builder Loan                                                                                                                                 637
Debt Consolidation Loan                                                                                                                             620
Student Loan                                                                                                                                        618
                                                                                                                                                   ... 
Payday Loan, Debt Consolidation Loan, Mortgage Loan, and Student Loan                   

In [111]:
# Remove bad interest rates

df_train = df_train[df_train.Interest_Rate < 50]
print(df_train['Interest_Rate'].value_counts())

5     1990
8     1928
6     1925
12    1919
10    1882
18    1860
9     1846
15    1811
20    1754
7     1752
17    1727
11    1717
16    1679
19    1631
3     1107
1     1046
13    1040
14     996
2      995
4      987
30     866
24     843
32     831
29     830
23     827
28     807
22     796
21     774
34     766
27     755
25     727
31     705
26     693
33     681
Name: Interest_Rate, dtype: int64


In [112]:
print(df_train['Payment_Behaviour'].value_counts())

Low_spent_Small_value_payments      10774
High_spent_Medium_value_payments     7593
Low_spent_Medium_value_payments      5831
High_spent_Large_value_payments      5786
High_spent_Small_value_payments      4897
Low_spent_Large_value_payments       4451
!@9#%8                               3161
Name: Payment_Behaviour, dtype: int64


In [113]:
# Deal with categories

df = pd.get_dummies(df_train['Payment_Behaviour'], prefix='Payment_Behaviour')
df_train = pd.concat([df_train, df], axis=1).drop('Payment_Behaviour', axis=1)

df = pd.get_dummies(df_train['Occupation'], prefix='Occupation')
df_train = pd.concat([df_train, df], axis=1).drop('Occupation', axis=1)

df = pd.get_dummies(df_train['Type_of_Loan'], prefix='Type_of_Loan')
df_train = pd.concat([df_train, df], axis=1).drop('Type_of_Loan', axis=1)

df = pd.get_dummies(df_train['Payment_of_Min_Amount'], prefix='Payment_of_Min_Amount')
df_train = pd.concat([df_train, df], axis=1).drop('Payment_of_Min_Amount', axis=1)


In [114]:
# Deal with ordinal variables

## Ordinal - Credit_Mix, Credit_Score

In [115]:
print(df_train['Credit_Mix'].value_counts())

Standard    15568
Good         9260
Bad          9032
_            8633
Name: Credit_Mix, dtype: int64


In [116]:
Credit_Mix_mapper = {'Bad':0, 'Standard':1, 'Good':2, '_':0}
df_train['Credit_Mix'] = df_train['Credit_Mix'].replace(Credit_Mix_mapper)

In [117]:
print(df_train['Credit_Score'].value_counts())

Standard    22481
Poor        13173
Good         6839
Name: Credit_Score, dtype: int64


In [118]:
Credit_Score_mapper = {'Poor':0, 'Standard':1, 'Good':2}
df_train['Credit_Score'] = df_train['Credit_Score'].replace(Credit_Score_mapper)

In [122]:
df_train.Credit_History_Age.describe()


count                     42493
unique                      404
top       18 Years and 4 Months
freq                        210
Name: Credit_History_Age, dtype: object

In [136]:


import regex as re

df_train.Credit_History_Age = df_train.Credit_History_Age.str.extract('(\d+)').astype(int)

In [137]:
df_train.Credit_History_Age.astype(str).describe()

count     42493
unique       34
top          18
freq       2227
Name: Credit_History_Age, dtype: object